Задание второе: решить систему, реализовав итерационные методы, указанные в варианте (варианты - 12.7 по методичке, стр.101, матрицы - из предыдущего задания, 11.6, стр.94)

In [1]:
import numpy as np

# 1 вариант

№ 1
Найти решение $x^{*}$ методом Гаусса

In [2]:
# Матрица из 2 варианта 11.6

A = np.array([
    [7.35272, 0.88255, -1.378574],
    [0.88255,   5.58351,   0.528167],
    [-2.27005,  0.528167,   4.430329]
])

b = np.array([
    [1],
    [0],
    [0]
])

In [3]:
# По умолчанию мат пакет решает методом Гаусса

x_star = np.linalg.solve(A, b)

In [4]:
x_star

array([[ 0.15557738],
       [-0.03249829],
       [ 0.0835904 ]])

№ 2
Преобразовать исходную систему к системе вида $x = H_Dx+g_D$ , где $H_D = E − D^{-1}A$, $g_D = D^{-1} * b$.
 Здесь D — диагональная матрица, у которой на диагонали находятся диагональные элементы матрицы A.
 Вычислить $||H_D||_{∞}$.

In [5]:
n = A.shape[0]

In [6]:
D = np.identity(n)
D[np.diag_indices_from(D)] = np.diag(A)
D

array([[7.35272 , 0.      , 0.      ],
       [0.      , 5.58351 , 0.      ],
       [0.      , 0.      , 4.430329]])

In [7]:
H_D = np.identity(n) - np.matmul(np.linalg.inv(D), A)
H_D

array([[ 0.        , -0.12003041,  0.1874917 ],
       [-0.15806366,  0.        , -0.09459408],
       [ 0.51238858, -0.1192162 ,  0.        ]])

In [8]:
g_D = np.matmul(np.linalg.inv(D), b)
g_D

array([[0.13600409],
       [0.        ],
       [0.        ]])

In [9]:
H_D_norm = np.linalg.norm(H_D, ord=np.inf)
H_D_norm

0.6316047860102488

№ 3
Вычислить априорную оценку погрешности $||x^{(7)} − x^∗||_∞$

In [10]:
def inf_norm(arr):
    return np.linalg.norm(arr, ord=np.inf)

x_0 = np.ones_like(g_D)
k = 7

estimate = inf_norm(H_D) ** k * inf_norm(x_0) + (inf_norm(H_D) ** k * inf_norm(g_D)) / (1 - inf_norm(H_D))
estimate

0.05490076369097396

In [11]:
np.matmul(H_D, np.matmul(H_D, x_0) + g_D) + g_D

array([[ 0.24004726],
       [-0.06935226],
       [ 0.13437424]])

№ 4
Вычислить приближение $x^{(7)}$ методом простой итерации. Вывести его фактическую
погрешность, апостериорную оценку, априорную оценку. Уточнить последнее приближение по Люстернику. Вывести его фактическую погрешность.

In [12]:
def simple_iter(H, g, k: int):
    appr = {}
    x = np.zeros((n, 1))
    for i in range(k):
        x = np.matmul(H, x) + g
        appr[i + 1] = x
    return appr

In [13]:
appr = simple_iter(H_D, g_D, 7)
x = appr[k]
posteriori_est = (inf_norm(H_D) * inf_norm(appr[k] - appr[k - 1])) / (1 - inf_norm(H_D))
actual_error = inf_norm(x - x_star)
print(f'Априорная оценка: {estimate}\n'
      f'Апостериорная оценка: {posteriori_est}\n'
      f'Фактическая погрешность: {actual_error}\n')

Априорная оценка: 0.05490076369097396
Апостериорная оценка: 0.00044841226065225166
Фактическая погрешность: 0.00022110801716208883



In [14]:
# Проверим спектральный радиус матрицы H

def spec_rad(M):
    return np.max(np.abs(np.linalg.eigvals(M)))

p_H = spec_rad(H_D)
p_H

0.387849854520505

In [15]:
# Спектральный радиус меньше 1, можно применять метод Люстерника

x_lust = appr[k - 1] + (appr[k] - appr[k - 1]) / (1 - p_H)
inf_norm(x_star - x_lust)

9.089454243642325e-05

№ 5
Вычислить приближение $x^{(7)}$ к решению системы $x=H_{D}x+g_D$ методом Зейделя.
Вывести его фактическую погрешность.
Сравнить с решением, полученным методом простой итерации.

In [16]:
def seidel_method(H, g, k) -> list:
    n = H.shape[0]
    prev_x = np.zeros(n)
    x = np.zeros(n)
    for it in range(k):
        for i in range(n):
            x[i] = np.dot(H[i, 0:i], x[0:i]) + np.dot(H[i, i:n], prev_x[i:n]) + g[i]
        prev_x = x
    return x.reshape((n, 1))

In [17]:
x_seid = seidel_method(H_D, g_D, 7)
print(f'Факт. погрешность методом простой итерации: {inf_norm(x - x_star)}')
print(f'Факт. погрешность методом Зейделя {inf_norm(x - x_seid)}')

Факт. погрешность методом простой итерации: 0.00022110801716208883
Факт. погрешность методом Зейделя 0.00022086965703820016


№ 6
При выполнении задания в математическом пакете определить спектральный радиус матрицы перехода,
если рассматривать метод Зейделя как метод простой итерации

In [18]:
from numpy.linalg import inv

def get_seid_matrices(H, g):
    H_L = np.tril(H, k=-1)
    H_R = np.triu(H)
    E = np.identity(H.shape[0])

    left_term = inv(E - H_L)
    H_seid = np.matmul(left_term, H_R)
    g_seid = np.matmul(left_term, g)

    return H_seid, g_seid

In [19]:
H_seid, g_seid = get_seid_matrices(H_D, g_D)
print(f"||H|| = {inf_norm(H_seid)}")
print(f"p(H) = {spec_rad(H_seid)}")

||H|| = 0.3075221142651971
p(H) = 0.16509089667328342


№ 7
Вычислить приближение $x^{(7)}$ методом верхней релаксации.

In [20]:
def successive_over_relaxation(H, g, k):
    q = 2 / (1 + np.sqrt(1 - spec_rad(H) ** 2))
    n = H.shape[0]
    prev_x = np.zeros(n)
    x = np.zeros(n)
    for it in range(k):
        for i in range(n):
            x[i] = prev_x[i] + q * (np.dot(H[i, 0:i], x[0:i]) + np.dot(H[i, (i+1):n], x[(i+1):n]) - prev_x[i] + g[i])
        prev_x = x
    return x.reshape((n, 1))

In [28]:
x_relax = successive_over_relaxation(H_D, g_D, 7)
errors = [
    (inf_norm(x_star - x), 'Метод простой итерации'),
    (inf_norm(x_star - x_lust), 'Уточнение методом Люстерника'),
    (inf_norm(x_star - x_seid), 'Метод Зейделя'),
    (inf_norm(x_star - x_relax), 'Метод верхней релаксации')
]
errors = list(sorted(errors, key=lambda x: x[0], reverse=True))
print('Фактические погрешности (в порядке убывания):\n')
print('\n'.join([f'{er[1]} : {er[0]}' for er in errors]))

Фактические погрешности (в порядке убывания):

Метод простой итерации : 0.00022110801716208883
Уточнение методом Люстерника : 9.089454243642325e-05
Метод Зейделя : 4.180425676458821e-07
Метод верхней релаксации : 8.061693618222066e-09
